## ZINC Visualization with Init PE

In [ ]:
import os
os.chdir('../') # go to root folder of the project
print(os.getcwd())

In [ ]:
import torch
import networkx as nx
from itertools import count
import matplotlib.pyplot as plt
import numpy as np
from itertools import count

In [ ]:
from data.data import LoadData
zinc_d = LoadData('ZINC')

pos_enc_dim = 24
zinc_d._init_positional_encodings(pos_enc_dim, 'rand_walk')
zinc_d._add_eig_vecs(36)

In [ ]:
num_nodes = []
num_unique_RWPEs = []
num_unique_LapPEs = []

for g_ in zinc_d.val:
    num_nodes.append(g_[0].number_of_nodes())
    num_unique_RWPEs.append(len(torch.unique(g_[0].ndata['pos_enc'], dim=0)))
for g_ in zinc_d.val:
    num_unique_LapPEs.append(len(torch.unique(g_[0].ndata['eigvec'], dim=0)))

In [ ]:
def plot_init_PE_comparison(num_nodes, num_unique_PEs, PE='RWPE'):
    fig = plt.figure(dpi=100, figsize=(7, 6))
    ax = plt.axes()
    plt.xlabel("Number of nodes", fontsize=10)
    if PE == 'RWPE':
        plt.title("ZINC val (1K): Comparison of no. of nodes v/s no. of unique RWPEs", fontsize=15)
        plt.ylabel("Number of unique RWPEs", fontsize=10)
    elif PE == 'LapPE':
        plt.title("ZINC val (1K): Comparison of no. of nodes v/s no. of unique LapPEs", fontsize=15)
        plt.ylabel("Number of unique LapPEs", fontsize=10)
    x = np.array(num_nodes)
    y1 = np.array(num_unique_PEs)
    #plt.xticks(x)
    plt.hist2d(x, y1, (50, 50), cmap=plt.cm.Reds)# plt.cm.jet)
    plt.colorbar()
    plt.xlim([10, 35])
    plt.ylim([10, 35])
    #plt.scatter(x, y1, marker="o", color="green", linewidth=0.5)
    x = np.linspace(9,40,100)
    y = x
    plt.plot(x, y, '-r', )

    #fig.savefig('out_ZINC_PE_viz/ZINC_valset_'+PE+'.pdf', bbox_inches='tight') 
    plt.show()

In [ ]:
plot_init_PE_comparison(num_nodes, num_unique_RWPEs, 'RWPE')
plot_init_PE_comparison(num_nodes, num_unique_LapPEs, 'LapPE')

In [ ]:
graph_ids = [212,672]   # not equal
graph_ids = [91,967] # equal

In [ ]:
for idx in graph_ids:
    fig = plt.figure(dpi=100, figsize=(8, 6))
    g_zinc_trial = zinc_d.val[idx][0]
    g = g_zinc_trial.to_networkx(node_attrs=['pos_enc'])
    #groups = set(nx.get_node_attributes(g,'pos_enc').values())
    groups = torch.unique(g_zinc_trial.ndata['pos_enc'],dim=0)
    mapping = dict(zip(groups,count()))
    nodes = g.nodes()
    colors = []
    for n in nodes:
        for key in mapping.keys():
            if torch.equal(key, g.nodes[n]['pos_enc']):
                color = mapping[key]
        colors.append(color)
        
    pos = nx.spring_layout(g)
    ec = nx.draw_networkx_edges(g, pos, alpha=0.2)
    nc = nx.draw_networkx_nodes(g, pos, nodelist=nodes, node_color=colors, 
                                node_size=100, cmap=plt.cm.jet)
    plt.colorbar(nc)
    #plt.xlabel("ZINC Val id: " +str(idx), fontsize=16)
    plt.title("nodes: "+ str(num_nodes[idx]) + " | unique RWPEs: "+str(num_unique_RWPEs[idx]), fontsize=16)
    #fig.savefig('out_ZINC_PE_viz/ZINC_valset_graph_'+str(idx)+'.pdf', bbox_inches='tight') 
    plt.show()